In [21]:
# we set a counter to register the same dataset again under a different name when we are debugging
global regis_num 
regis_num = 0

In [22]:
!pip install pyyaml==5.1
#!pip uninstall torch
!yes | pip uninstall detectron2
#!pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
    
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Found existing installation: detectron2 0.6+cu111
Uninstalling detectron2-0.6+cu111:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/detectron2-0.6+cu111.dist-info/*
    /usr/local/lib/python3.8/dist-packages/detectron2/*
    /usr/local/lib/python3.8/dist-packages/tools/*
Proceed (y/n)?   Successfully uninstalled detectron2-0.6+cu111
yes: standard output: Broken pipe
Looking in links: https://download.pytorch.org/whl/torch_stable.html
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
  Using cached https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/detectron2-0.6%2Bcu111-cp38-cp38-linux_x86_64.whl (6.9 MB)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [23]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
torch:  1.9 ; cuda:  cu111
detectron2: 0.6


In [24]:
#!pip install datetime

     |████████████████████████████████| 52 kB 1.6 MB/s eta 0:00:011
     |████████████████████████████████| 249 kB 6.4 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [25]:
import os
import sys

In [26]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data import build_detection_train_loader
from detectron2.data import DatasetMapper
from detectron2.data import DatasetCatalog

from detectron2.config import get_cfg
from detectron2 import model_zoo

In [27]:
import json

In [28]:
# Error Message while training: module 'distutils' has no attribute 'version'
#!python3 -m pip install setuptools==58.2.0
# then restart the kernel

In [29]:
## Function to register our datasets 
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

dataset_name_train = "small_thesis_train"
dataset_name_val = "small_thesis_val"

def register(name, path_labels, path_data):

    # Check if dataset name already exists in catalog
    if name in DatasetCatalog.list():
        # Remove existing dataset from catalog
        DatasetCatalog.remove(name)
        MetadataCatalog.remove(name)

    # Register your dataset using register_coco_instances
    register_coco_instances(name, {}, path_labels, path_data)

# if it throws an error that name is already registered, just change the name... 
register(dataset_name_train, "/workspace/data/labels_train.json", "/workspace/data/data_030/train/data")
register(dataset_name_val, "/workspace/data/labels_val.json", "/workspace/data/data_030/val/data")

In [30]:
# register datasets

dataset_name_train = "thesis_030_train"
dataset_name_val = "thesis_030_val"

register_coco_instances(dataset_name_train,{}, "/workspace/data/labels_train.json", "/workspace/data/data_030/train/data")
register_coco_instances(dataset_name_val,{}, "/workspace/data/labels_val.json", "/workspace/data/data_030/val/data")


In [31]:
base_path = "/workspace/scripts"

os.chdir(base_path)

In [32]:
# Taken from https://eidos-ai.medium.com/training-on-detectron2-with-a-validation-set-and-plot-loss-on-it-to-avoid-overfitting-6449418fbf4e
# https://gist.github.com/ortegatron/c0dad15e49c2b74de8bb09a5615d9f6b

from EvalLossHook import LossEvalHook


class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)
                     
    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            self.cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        return hooks

    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg)
 


In [33]:
# Lukas Univ Edingbourgh Trainer Version 

class OurTrainer(DefaultTrainer):

    def __init__(self, cfg):
        super().__init__(cfg)

        if isinstance(cfg.DATASETS.EVAL, str):
            self.eval_datasets = [cfg.DATASETS.EVAL]
        else:
            self.eval_datasets = cfg.DATASETS.EVAL

        # prepare evaluation
        self.eval_loaders = []
        self.evaluators = []
        for dataset in self.eval_datasets:

            loader = build_detection_test_loader(DatasetCatalog.get(dataset), 
                                                 mapper=DatasetMapper(cfg, is_train=False))

            self.eval_loaders.append(loader)
            self.evaluators.append(COCOEvaluator(
                dataset,
                output_dir=cfg.OUTPUT_DIR 
                ))
    
    def build_train_loader(cls, cfg):
        if cfg.SOLVER.STRONG_AUGMENT:
            print("STRONG AUGMENTATIONS TO TRAINING SET")
            return build_detection_train_loader(cfg, mapper=DatasetMapperAugment(cfg, is_train=True)) 
        else:
            return build_detection_train_loader(cfg)

    
    def after_step(self):
        super().after_step()

        if (self.iter+1) % self.cfg.TEST.INTERVAL == 0:                                   
    
            for dataset, loader, evaluator in zip(self.eval_datasets, 
                                                    self.eval_loaders,
                                                    self.evaluators):
    
                results = inference_on_dataset(self.model,
                                                loader,
                                                evaluator)
                with open(
                    os.path.join(
                        self.cfg.OUTPUT_DIR,
                        'eval_'+dataset+'_iter_'+str(self.iter)+'.json'),
                        'w') as out:
                    json.dump(results, out)

In [36]:
# set up the model for training

from detectron2.engine import DefaultTrainer
from datetime import datetime

now = datetime.now() 

output_dir = f"/workspace/output/output_new_res030_{now.strftime('%m.%d.%H.%M')}"

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.OUTPUT_DIR = output_dir

cfg.DATASETS.TRAIN = (dataset_name_train)
cfg.DATASETS.TEST = (dataset_name_val,)

cfg.DATALOADER.NUM_WORKERS = 2

cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[4, 8, 16, 32, 64]]
cfg.SOLVER.MAX_ITER = 500
cfg.TEST.EVAL_PERIOD = 200 # für unseren Evaluator
#cfg.TEST.INTERVAL = 400 # für Lukas Evaluator
#cfg.SOLVER.STEPS = (2000, 2500)
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [37]:
print(f"FIND EXPERIMENT RESULTS AT \n {cfg.OUTPUT_DIR}")

FIND EXPERIMENT RESULTS AT 
 /workspace/output/output_new_res030_03.30.13.51


In [38]:
# start the training

trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[03/30 13:52:06 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

WARNING [03/30 13:52:06 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/30 13:52:06 d2.data.datasets.coco]: Loaded 11696 images in COCO format from /workspace/data/labels_train.json
[03/30 13:52:06 d2.data.build]: Removed 0 images with no usable annotations. 11696 images left.
[03/30 13:52:07 d2.data.build]: Distribution of instances among all 3 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|     DT     | 53154        |     OT     | 103          |     TT     | 1905         |
|            |              |            |              |            |              |
|   total    | 55162        |            |              |            |              |
[03/30 13:52:07 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 70

Some model parameters or buffers are not found in the checkpoint:
backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
The checkpoint state_dict contains keys that are not used by the model:
  fc1000.{bias, weight}
  stem.conv1.bias


[03/30 13:52:09 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.8/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[03/30 13:52:17 d2.utils.events]:  eta: 0:02:53  iter: 19  total_loss: 1.024  loss_cls: 0.1302  loss_box_reg: 0.001226  loss_rpn_cls: 0.7036  loss_rpn_loc: 0.1259  time: 0.3587  data_time: 0.0683  lr: 0.00077924  max_mem: 5394M
[03/30 13:52:24 d2.utils.events]:  eta: 0:02:46  iter: 39  total_loss: 0.8309  loss_cls: 0.141  loss_box_reg: 0.002363  loss_rpn_cls: 0.6006  loss_rpn_loc: 0.07201  time: 0.3608  data_time: 0.0604  lr: 0.0015784  max_mem: 5394M
[03/30 13:52:31 d2.utils.events]:  eta: 0:02:39  iter: 59  total_loss: 0.4876  loss_cls: 0.05479  loss_box_reg: 0.001844  loss_rpn_cls: 0.3457  loss_rpn_loc: 0.06985  time: 0.3582  data_time: 0.0591  lr: 0.0023776  max_mem: 5394M
[03/30 13:52:38 d2.utils.events]:  eta: 0:02:32  iter: 79  total_loss: 0.4271  loss_cls: 0.05595  loss_box_reg: 0.01025  loss_rpn_cls: 0.2663  loss_rpn_loc: 0.06889  time: 0.3571  data_time: 0.0630  lr: 0.0031768  max_mem: 5394M
[03/30 13:52:45 d2.utils.events]:  eta: 0:02:25  iter: 99  total_loss: 0.3972  loss_c

[03/30 13:55:43 d2.utils.events]:  eta: 0:01:27  iter: 259  total_loss: 0.6472  loss_cls: 0.1941  loss_box_reg: 0.2289  loss_rpn_cls: 0.1304  loss_rpn_loc: 0.08655  validation_loss: 0.5844  time: 0.3572  data_time: 0.0594  lr: 0.01037  max_mem: 5394M
[03/30 13:55:50 d2.utils.events]:  eta: 0:01:19  iter: 279  total_loss: 0.6474  loss_cls: 0.2088  loss_box_reg: 0.2469  loss_rpn_cls: 0.1304  loss_rpn_loc: 0.05432  validation_loss: 0.5844  time: 0.3576  data_time: 0.0626  lr: 0.011169  max_mem: 5394M
[03/30 13:55:57 d2.utils.events]:  eta: 0:01:12  iter: 299  total_loss: 0.6944  loss_cls: 0.2048  loss_box_reg: 0.2144  loss_rpn_cls: 0.1097  loss_rpn_loc: 0.0927  validation_loss: 0.5844  time: 0.3580  data_time: 0.0621  lr: 0.011968  max_mem: 5394M
[03/30 13:56:05 d2.utils.events]:  eta: 0:01:05  iter: 319  total_loss: 0.7218  loss_cls: 0.2196  loss_box_reg: 0.2439  loss_rpn_cls: 0.1092  loss_rpn_loc: 0.06089  validation_loss: 0.5844  time: 0.3583  data_time: 0.0638  lr: 0.012767  max_mem: 

[03/30 13:59:03 d2.utils.events]:  eta: 0:00:07  iter: 479  total_loss: 0.6794  loss_cls: 0.2212  loss_box_reg: 0.2586  loss_rpn_cls: 0.1  loss_rpn_loc: 0.06771  validation_loss: 0.6494  time: 0.3599  data_time: 0.0622  lr: 0.019161  max_mem: 5394M
[03/30 14:00:16 d2.utils.events]:  eta: 0:00:00  iter: 499  total_loss: 0.6766  loss_cls: 0.2248  loss_box_reg: 0.241  loss_rpn_cls: 0.1135  loss_rpn_loc: 0.08935  validation_loss: 0.7143  time: 0.3602  data_time: 0.0656  lr: 0.01996  max_mem: 5394M
[03/30 14:00:16 d2.engine.hooks]: Overall training speed: 498 iterations in 0:02:59 (0.3602 s / it)
[03/30 14:00:16 d2.engine.hooks]: Total training time: 0:08:05 (0:05:06 on hooks)
WARNING [03/30 14:00:16 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/30 14:00:16 d2.data.datasets.coco]: Loaded 1801 images in COCO format from /workspace/data/labels_val.json
[03/30 14:00:16 d2.data.dataset_mapper]: [DatasetMapper] Augmentations

In [44]:
# put it all in a function to be able to build a loop for the different resolutions

def train_detectron_standard(dataset_name_train, dataset_name_val, resolution):
    
    now = datetime.now() 
    output_dir = f"/workspace/output/output_new_res{resolution}_{now.strftime('%m.%d.%H.%M')}"

    cfg = get_cfg()
    
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
    cfg.OUTPUT_DIR = output_dir

    cfg.DATASETS.TRAIN = (dataset_name_train)
    cfg.DATASETS.TEST = (dataset_name_val,)

    cfg.DATALOADER.NUM_WORKERS = 2

    cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[4, 8, 16, 32, 64]]
    cfg.SOLVER.MAX_ITER = 30000
    cfg.TEST.EVAL_PERIOD = 5000 # für unseren Evaluator
    #cfg.TEST.INTERVAL = 400 # für Lukas Evaluator
    #cfg.SOLVER.STEPS = (2000, 2500)
    cfg.SOLVER.IMS_PER_BATCH = 8
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    
    trainer = MyTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()
    
    predictor = DefaultPredictor(cfg)
    
    return cfg, trainer, predictor

In [ ]:
# train the model for all our resolutions

resolution_set = ['030', '035','040','045','050','070','100']

for res in resolution_set:

    #define the names 
    dataset_name_train = "tower_train_" + str(res) + "1"
    dataset_name_val = "tower_val_" + str(res) + "1"

    try: 
        # if it throws an error that name is already registered, just change the name... 
        register_coco_instances(dataset_name_train, {}, "/workspace/data/labels_train.json", "/workspace/data/data_"+str(res)+"/train/data")
        register_coco_instances(dataset_name_val, {}, "/workspace/data/labels_val.json", "/workspace/data/data_"+str(res)+"/val/data")

        #MetadataCatalog.get(dataset_name_train).set(thing_classes=["distribution", "transmission"])
        #MetadataCatalog.get(dataset_name_val).set(thing_classes=["distribution", "transmission"])
    
    except: 
        regis_num += 1
        dataset_name_train += str(regis_num)
        dataset_name_val += str(regis_num)

        # if it throws an error that name is already registered, just change the name... 
        register_coco_instances(dataset_name_train, {}, "/workspace/data/labels_train.json", "/workspace/data/data_"+str(res)+"/train/data")
        register_coco_instances(dataset_name_val, {}, "/workspace/data/labels_val.json",  "/workspace/data/data_"+str(res)+"/val/data")
        #MetadataCatalog.get(dataset_name_train).set(thing_classes=["distribution", "transmission"])
        #MetadataCatalog.get(dataset_name_val).set(thing_classes=["distribution", "transmission"])

    configuration, train_obj, predictor_obj = train_detectron_standard(dataset_name_train, dataset_name_val, res)


[03/29 15:14:00 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

WARNING [03/29 15:14:00 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/29 15:14:00 d2.data.datasets.coco]: Loaded 11696 images in COCO format from /workspace/data/labels_train.json
[03/29 15:14:00 d2.data.build]: Removed 0 images with no usable annotations. 11696 images left.
[03/29 15:14:01 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[03/29 15:14:01 d2.data.build]: Using training sampler TrainingSampler
[03/29 15:14:01 d2.data.common]: Serializing 11696 elements to byte tensors and concatenating them all ...
[03/29 15:14:01 d2.data.common]: Serialized dataset takes 5.84 MiB
WARNING [03/29 15:14:01 d2.solver.build]: SOLVER.STEPS contains values larger than SOLVER.MAX_ITER. These values will be ignored.
[03/29 15:14:01 d2.data.dataset_mapper]: [DatasetMapper] Augmenta

Some model parameters or buffers are not found in the checkpoint:
backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
The checkpoint state_dict contains keys that are not used by the model:
  fc1000.{bias, weight}
  stem.conv1.bias


[03/29 15:14:01 d2.engine.train_loop]: Starting training from iteration 0
[03/29 15:14:09 d2.utils.events]:  eta: 3:01:14  iter: 19  total_loss: 0.9213  loss_cls: 0.1291  loss_box_reg: 0.004402  loss_rpn_cls: 0.6921  loss_rpn_loc: 0.093  time: 0.3608  data_time: 0.0771  lr: 0.00039962  max_mem: 6032M
[03/29 15:14:16 d2.utils.events]:  eta: 3:00:39  iter: 39  total_loss: 0.8399  loss_cls: 0.1261  loss_box_reg: 0.004666  loss_rpn_cls: 0.6078  loss_rpn_loc: 0.07499  time: 0.3590  data_time: 0.0653  lr: 0.00079922  max_mem: 6033M
[03/29 15:14:23 d2.utils.events]:  eta: 3:00:34  iter: 59  total_loss: 0.6104  loss_cls: 0.05824  loss_box_reg: 0.00354  loss_rpn_cls: 0.4561  loss_rpn_loc: 0.06981  time: 0.3597  data_time: 0.0662  lr: 0.0011988  max_mem: 6033M
[03/29 15:14:30 d2.utils.events]:  eta: 3:00:51  iter: 79  total_loss: 0.4779  loss_cls: 0.05255  loss_box_reg: 0.005566  loss_rpn_cls: 0.3075  loss_rpn_loc: 0.1133  time: 0.3614  data_time: 0.0662  lr: 0.0015984  max_mem: 6033M
[03/29 15:

## visualize predictions

In [40]:

from detectron2.utils.visualizer import ColorMode

import random
import cv2
import matplotlib.pyplot as plt

def plot_sample(dataset_name, n=1):
    dataset_custom = DatasetCatalog.get(dataset_name)
    dataset_custom_metadata = MetadataCatalog.get(dataset_name)
    
    for s in random.sample(dataset_custom, n):
        img = cv2.imread(s["file_name"])
        v = Visualizer(img[:, :, ::-1], metadata=dataset_custom_metadata, scale=0.5)
        v = v.draw_dataset_dict(s)
        plt.figure(figsize=(15,20))
        plt.imshow(v.get_image())
        plt.imshow()

In [42]:
val_or_train = "train" # write a string here (either "val" or "train")
location = ""
predict_path = '/workspace/data'
dataset_dicts = json.load(open(f'{predict_path}/labels_{val_or_train}.json'))

for d in random.sample(dataset_dicts['images'], 3):  
    im = cv2.imread(f'{base_path}/data_030/{val_or_train}/data/{d["file_name"]}')

    outputs = predictor_obj(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata= MetadataCatalog.get("tower_val1"), 
                   scale=0.5 
                   #,instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow(out.get_image()[:, :, ::-1])

NameError: name 'predictor_obj' is not defined

In [ ]:
## inspect predictions

In [36]:
import cv2

In [43]:
im = cv2.imread("/workspace/data/data_030/val/data/AR_3258.1283105585248_669.62722839057085_166.png")

In [44]:
from detectron2.engine import DefaultPredictor

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [45]:
outputs = predictor(im)

In [46]:
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [ ]:
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TEST[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imshow("some", out.get_image()[:, :, ::-1])

In [52]:
?cv2.imshow